# Introducción
Este proyecto tiene como objetivo desarrollar un modelo de deep learning capaz de clasificar recetas en categorías basadas en sus ingredientes. La idea es facilitar la búsqueda de recetas al predecir el tipo de cocina al que pertenece una lista de ingredientes.

El proyecto abordará esta tarea implementando un modelo que procese ingredientes, los interprete en el contexto de distintas cocinas, y aprenda a clasificar la receta en categorías como "italiana", "mexicana" o "asiática". Para lograrlo, primero se recolectará un dataset de recetas, se preprocesarán los datos para convertir los ingredientes en vectores utilizando un embedding, y se entrenará un modelo de Red Neuronal Recurrente (RNN) para clasificar las recetas.

---

# Exploración, explicación y limpieza de datos
## Explicación de los datos
Para comenzar, se encontró un dataset en Kaggle que contiene recetas de distintas cocinas. El dataset se llama "What's Cooking?" y se divide en tres archivos. Primeramente, el archivo `train.json` contiene las recetas de entrenamiento, cada una con una lista de ingredientes y una etiqueta que indica el tipo de cocina. Por otro lado, el archivo `test.json` contiene recetas de prueba, pero sin las etiquetas de cocina, ya que este kaggle está diseñado para competencias. 

Por el motivo anterior, se decidió dividir el archivo `train.json` en dos partes: una para entrenamiento y otra de testeo. De esta manera, se podrá evaluar el modelo con datos que no ha visto antes.

El dataset se creo en el 2015 por el mismo Kaggle y aquí está el [link](https://www.kaggle.com/competitions/whats-cooking/data).

In [108]:
import pandas as pd
import json

# Cargar los archivos JSON de entrenamiento y prueba
with open('whats-cooking/train.json') as f:
    data = json.load(f)
df = pd.DataFrame(data)

df['cuisine'] = df['cuisine'].astype('category')

# Dividir el conjunto en entrenamiento y prueba
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print('Entrenamiento: ', train_df.shape)
print('Prueba: ', test_df.shape)

print(train_df.head())
print(test_df.head())

Entrenamiento:  (31819, 3)
Prueba:  (7955, 3)
          id   cuisine                                        ingredients
23436  46505   mexican  [shredded cheddar cheese, chicken meat, choppe...
7901   16624    indian  [fresh cilantro, purple onion, ground coriande...
25718   3415  filipino  [sugar, garlic, onions, vinegar, green chilies...
16909   4589  moroccan  [raw pistachios, purple onion, couscous, dried...
34830   7766   mexican  [tomatoes, pepper, salsa, sliced green onions,...
          id  cuisine                                        ingredients
21513   7958  chinese  [pork, cooking oil, bamboo shoots, chinese ric...
1796   36179  spanish  [hog casings, hungarian paprika, ancho powder,...
21861   8331    greek  [lamb stock, lemon, lamb shoulder, onions, gro...
26571  41097   indian  [green peas, cinnamon sticks, clove, chopped o...
28720   2007  italian  [vegetable oil spray, cumin seed, grated parme...


## Análisis de los datos

Ya revisando concretamente cada uno de los archivos, se puede observar que el df `train_df` contiene 31819 recetas de entrenamiento, cada una con los siguientes campos:

|   # | Column      | Non-Null Count | Dtype    |
|-----|-------------|----------------|----------|
| 0   | id          | 31819 non-null | int64    |
| 1   | cuisine     | 31819 non-null | category |
| 2   | ingredients | 31819 non-null | object   |

Por otro lado, el df `test_df` contiene 7955 recetas de prueba, cada una con los siguientes campos:

|   # | Column      | Non-Null Count | Dtype    |
|-----|-------------|----------------|----------|
| 0   | id          | 7955 non-null | int64    |
| 1   | cuisine     | 7955 non-null | category |
| 2   | ingredients | 7955 non-null | object   |

In [110]:
# Calcular la cantidad de ingredientes por receta en el conjunto de entrenamiento
train_df['num_ingredients'] = train_df['ingredients'].apply(len)

# Calcular el promedio de ingredientes por receta
avg_num_ingredients = train_df['num_ingredients'].mean()
print(f"Promedio de ingredientes por receta en el conjunto de entrenamiento: {avg_num_ingredients:.2f}")

# Calcular la cantidad de recetas por tipo de cocina
cuisine_counts = train_df['cuisine'].value_counts()
print(f"Número de cocinas en el conjunto de entrenamiento: {cuisine_counts.shape[0]}")
print(cuisine_counts)


# Calcular la cantidad de ingredientes únicos
unique_ingredients = set()
for ingredients in train_df['ingredients']:
    unique_ingredients.update(ingredients)
num_unique_ingredients = len(unique_ingredients)
print(f"Número de ingredientes únicos: {num_unique_ingredients}")

# Get the 5 recipes with the most ingredients
recipes_most_ingredients = train_df.loc[num_ingredients.nlargest(20).index]
print("Recipes with the most ingredients:")

# Print the quantity of ingredients for each recipe
for i, recipe in recipes_most_ingredients.iterrows():
    print(f"Recipe {i}: {len(recipe['ingredients'])} ingredients")


Promedio de ingredientes por receta en el conjunto de entrenamiento: 10.77
Número de cocinas en el conjunto de entrenamiento: 20
cuisine
italian         6271
mexican         5102
southern_us     3472
indian          2401
chinese         2163
french          2096
thai            1224
cajun_creole    1218
japanese        1139
greek            926
spanish          807
vietnamese       681
korean           664
moroccan         655
british          647
filipino         619
irish            516
jamaican         435
russian          400
brazilian        383
Name: count, dtype: int64
Número de ingredientes únicos: 6303
Recipes with the most ingredients:
Recipe 15289: 65 ingredients
Recipe 26103: 52 ingredients
Recipe 10513: 49 ingredients
Recipe 22906: 49 ingredients
Recipe 31250: 43 ingredients
Recipe 345: 40 ingredients
Recipe 6449: 40 ingredients
Recipe 3359: 40 ingredients
Recipe 10379: 38 ingredients
Recipe 294: 38 ingredients
Recipe 28480: 36 ingredients
Recipe 6139: 36 ingredients
Recip

## Limpieza de los datos

Haciendo una exploración de los datos, se puede observar que no hay valores nulos en los archivos `train.json` y `test.json`. Además, las columnas que contienen son muy directas para lo que se quiere realizar. Por lo tanto, no se realizará ninguna limpieza de datos.

Además, se identificaron 6703 ingredientes únicos en el dataset y 20 tipos de cocina distintos. Por otra parte, únicamente se encontraron 6303 ingredientes únicos en el dataset de entrenamiento, por lo que al momento de tokenizar los ingredientes, se deberá tener en cuenta que existen ingredientes en el dataset de prueba que no están presentes en el dataset de entrenamiento.

Otros datos relevantes es que, con diferencia, la cocina más común en el dataset es la italiana, seguida por la mexicana y la del sur de los Estados Unidos. Por otro lado, la cocina más rara en el dataset es la de Reino Unido.

## Transformación de los datos

Ahora que se ha explorado y limpiado los datos, se procederá a transformar los ingredientes en vectores utilizando un embedding. Para ello, primeramente se tokenizarán los ingredientes. Esto se hará utilizando la clase `Tokenizer` de Keras, la cual se encargará de convertir los ingredientes en secuencias de números enteros.

---

# Implementación del modelo

## Planteamiento del modelo

Ahora si, el plan que se tiene para realizar esta compleja tarea es realizar un modelo de aprendizaje profundo que se basa en embeddings de ingredientes generados mediante FastText. Esto se realiza con el objetivo de alimentar con estos embeddings a una red neuronal recurrente (RNN) para realizar la clasificación.

Importante recalcar que conforme se avance con el desarrollo del proyecto, se irá justificando cada decisión tomada y se explicará el proceso de implementación de cada componente del modelo. 

Finalmente, se realizarán pruebas posteriores con diferentes configuraciones de hiperparámetros y cambios en la arquitectura del modelo para buscar optimizar su rendimiento, de manera que se pueda obtener un modelo que sea capaz de clasificar recetas con la más alta precisión.

## Estructura del Proyecto

1. **Generación de Embeddings con FastText**: Creamos embeddings para los ingredientes de las recetas utilizando FastText. Este método permite capturar relaciones semánticas y contextuales entre los ingredientes, ayudando al modelo a entender similitudes y patrones entre diferentes combinaciones de ingredientes.
   
2. **Implementación de la RNN**: Utilizamos los embeddings generados como entrada a una red neuronal recurrente. La RNN es adecuada para este problema debido a su habilidad para manejar secuencias de longitud variable, lo cual es común en listas de ingredientes.

3. **Entrenamiento y Evaluación**: Entrenamos el modelo en un conjunto de datos de entrenamiento y lo evaluamos en un conjunto de prueba, con métricas de precisión y análisis del rendimiento.

A continuación, presentamos los detalles de la implementación de FastText, incluyendo las decisiones tomadas para optimizar los parámetros de este modelo.

In [113]:
from gensim.models import FastText

# Prepare data for FastText: each recipe's ingredients become a "sentence"
corpus = [list(map(str.lower, ingredients)) for ingredients in train_df['ingredients']]

# Train a FastText model
fasttext_model = FastText(
    sentences=corpus,
    vector_size=100,  # Dimension of the embeddings
    window=3,         # Context window size
    min_count=2,      # Ignores words with total frequency lower than this
    sg=1,             # 1 for skip-gram; 0 for CBOW
    epochs=10         # Number of training epochs
)

# Save model to disk (optional)
fasttext_model.save("ft_100_3_2_1_10.model")

# Example: Check similarity between specific ingredients
similarity = fasttext_model.wv.similarity("onion", "purple onion")
print(f"Similarity between 'onion' and 'purple onion': {similarity}")

# Example: Find most similar words to "tomato"
similar_words = fasttext_model.wv.most_similar("tomato", topn=5)
print("Most similar ingredients to 'tomato':", similar_words)


Similarity between 'onion' and 'purple onion': 0.7392172813415527
Most similar ingredients to 'tomato': [('tomatoes', 0.9098507761955261), ('sauce tomato', 0.8709294199943542), ('organic tomato', 0.85999596118927), ('yellow tomato', 0.8564296364784241), ('tomato chutney', 0.8525497317314148)]


### Generación de Embeddings de Ingredientes con FastText

Como se mencionó anteriormente, se realizó utilizando FastText para generar embeddings de los ingredientes. FastText es una biblioteca de código abierto desarrollada por Facebook que permite entrenar modelos de vectores de palabras y generar embeddings para palabras y frases. En este caso, utilizamos FastText para generar embeddings de los ingredientes de las recetas, lo cual nos permitirá representar los ingredientes en un espacio vectorial y alimentarlos a la red neuronal recurrente.

Aquí la razón por la que se utilizaron los parámetros de FastText:

- **`vector_size=100`**: Elegimos una dimensión de 100 para los embeddings, buscando un equilibrio entre precisión y eficiencia. Una dimensión mayor podría capturar más matices semánticos, pero también requiere mayor memoria y tiempo de cómputo.

- **`window=3`**: Este parámetro define el tamaño de la ventana de contexto alrededor de cada palabra. Optamos por 3 considerando que los ingredientes de un platillo suelen estar relacionados en términos cercanos (es decir, es probable que los ingredientes relacionados aparezcan juntos).

- **`min_count=2`**: Esto indica que FastText ignorará palabras con una frecuencia menor a 2. Este umbral permite reducir el ruido de ingredientes poco comunes, que no aportan gran valor al aprendizaje del modelo.

- **`sg=1`**: Al establecer `sg=1`, seleccionamos el modelo Skip-Gram en lugar del Continuous Bag of Words (CBOW). Skip-Gram es particularmente útil en este caso porque ayuda a capturar relaciones de palabras infrecuentes, lo cual es común en los ingredientes de recetas.

- **`epochs=10`**: Finalmente, usamos 10 épocas para entrenar el modelo. Este número de iteraciones asegura que el modelo tiene suficientes oportunidades para aprender relaciones complejas entre ingredientes sin sobreajustarse al conjunto de datos.

Estos parámetros de inicio se consideran adecuados para el problema de clasificación de recetas, se podrían ajustar pero al ser complicado evaluar el rendimiento de los embeddings, se decidió dejarlos así, será mejor dedicar esfuerzos en la optimización de la red neuronal.


In [114]:
import numpy as np

# Define la longitud máxima de la secuencia, para que todas las recetas tengan el mismo tamaño
max_len = 14  # Ajusta según la longitud promedio de ingredientes en tus recetas

# Cargar el modelo FastText
fasttext_model = FastText.load("ft_100_3_2_1_10.model")

def get_embedding_sequence(recipe, model, max_len=max_len):
    """
    Convierte una lista de ingredientes en una secuencia de embeddings.
    Rellena con vectores de ceros si la receta tiene menos de `max_len` ingredientes.
    """
    # Obtener embeddings de cada ingrediente si existe en el modelo
    embeddings = [model.wv[ingredient] for ingredient in recipe if ingredient in model.wv]
    # Recortar o completar con ceros según el tamaño `max_len`
    if len(embeddings) < max_len:
        embeddings.extend([np.zeros(model.vector_size)] * (max_len - len(embeddings)))
    else:
        embeddings = embeddings[:max_len]
    return np.array(embeddings)

# Generar secuencias de embeddings para todas las recetas en el conjunto de entrenamiento
X_train = np.array([get_embedding_sequence(recipe, fasttext_model) for recipe in train_df['ingredients']])


### Conversión de Ingredientes a Secuencias de Embeddings con FastText

Como se mencionó anteriormente, después de generar los embeddings de los ingredientes utilizando FastText, el siguiente paso es convertir cada receta en una secuencia de estos embeddings para alimentar al modelo de deep learning. Este proceso garantiza que todas las recetas tengan una representación consistente y adecuada para su procesamiento.

- **Definición de Longitud Máxima (`max_len=14`)**: Se define `max_len` como 14, indicando que cada receta debe estar representada por una secuencia de 14 vectores de embeddings. Si una receta tiene menos de 14 ingredientes, la secuencia se completará con vectores de ceros. Este valor se ajusta según la longitud promedio de ingredientes en las recetas.

- **Carga del Modelo FastText**: Se carga el modelo FastText previamente entrenado.

- **Función de Conversión a Secuencias de Embeddings**: La función definida convierte una lista de ingredientes en una secuencia de embeddings:
  - Se obtienen los embeddings para cada ingrediente si existe en el modelo.
  - Si la receta tiene menos de `max_len` ingredientes, se añaden vectores de ceros hasta completar la longitud deseada.
  - Si la receta tiene más de `max_len` ingredientes, se recorta la lista para que sólo los primeros `max_len` ingredientes se incluyan.
  - La función devuelve una matriz numpy con la secuencia de embeddings.

- **Generación de Secuencias de Embeddings para el Conjunto de Entrenamiento**: Se utiliza la función de conversión para transformar todas las recetas en el conjunto de entrenamiento en secuencias de embeddings, asegurando que los datos de entrada al modelo de deep learning estén preparados y estandarizados.

Este enfoque permite representar cada receta de una manera que la red neuronal pueda procesar eficazmente, facilitando la tarea de clasificación de recetas basada en sus ingredientes.

Además, es importante recalcar que es uno de los parametros que más se puede modificar y que podría alterar los resultados también, por lo que se realizarán pruebas posteriores para encontrar el mejor valor de `max_len`, buscando cual es el que mejor se ajusta a los datos.

In [115]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convertir las etiquetas de cocina a valores numéricos
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_df['cuisine'])
y_train = to_categorical(y_train_encoded)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking

# Definir los hiperparámetros
embedding_dim = 100  # Dimensionalidad de los embeddings que has usado con FastText
max_len = 14         # Longitud de la secuencia (igual a lo que definiste antes)
num_classes = y_train.shape[1]  # Número de clases, basado en la codificación one-hot de las etiquetas

# Construir el modelo RNN
model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_len, embedding_dim)),  # Ignora los ceros de relleno
    LSTM(128, return_sequences=False),  # Cambia a GRU si prefieres, y ajusta la cantidad de unidades
    Dropout(0.5),                       # Dropout para evitar overfitting
    Dense(64, activation='relu'),       # Capa densa intermedia
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Capa de salida con softmax para clasificación multiclas
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Implementación de una Red Neuronal Recurrente (RNN) para Clasificación de Recetas

Para este problema de clasificación de recetas basado en ingredientes, se ha optado por utilizar una Red Neuronal Recurrente (RNN). Las RNN son adecuadas para este tipo de problemas porque están diseñadas para trabajar con datos secuenciales, como texto o series temporales. En este caso, las secuencias de ingredientes pueden ser vistas como datos secuenciales donde el orden de los ingredientes y su contexto importan para la clasificación.

**Justificación del Uso de una RNN**:
- **Manejo de Secuencias**: Las RNN son excelentes para procesar secuencias de datos debido a su capacidad para mantener información contextual a través del tiempo.
- **Contexto de Ingredientes**: En una receta, el orden y combinación de ingredientes pueden ofrecer pistas significativas sobre el origen del platillo. Las RNN pueden capturar estas dependencias temporales y contextuales.

**Hiperparámetros del Modelo**:
- **`embedding_dim=100`**: Esta es la dimensionalidad de los embeddings que se usaron con FastText. Se ha mantenido esta misma dimensionalidad en la RNN para asegurar consistencia en la representación de los ingredientes.
- **`max_len=20`**: Se define la longitud de la secuencia como 20. Este valor se ha ajustado considerando la longitud promedio de ingredientes en las recetas, para asegurar que se capturen suficientes ingredientes sin añadir demasiados ceros de relleno.
- **`num_classes=y_train.shape[1]`**: Este valor define el número de clases, basado en la codificación one-hot de las etiquetas. Es importante que el modelo tenga el mismo número de salidas que clases en el problema de clasificación.

**Componentes del Modelo**:
- **`Masking(mask_value=0.0, input_shape=(max_len, embedding_dim))`**: Se utiliza una capa de `Masking` para ignorar los ceros de relleno en las secuencias, evitando que estos afecten el entrenamiento del modelo.
- **`LSTM(128, return_sequences=False)`**: Se elige una capa LSTM con 128 unidades. Las LSTM son un tipo de RNN que maneja mejor las dependencias a largo plazo. El número de unidades se ha seleccionado para equilibrar entre capacidad de aprendizaje y complejidad del modelo.
- **`Dropout(0.5)`**: Se añade una capa de Dropout con una tasa del 50% para reducir el riesgo de sobreajuste. El Dropout apaga aleatoriamente la mitad de las neuronas durante el entrenamiento, lo que mejora la generalización del modelo.
- **`Dense(64, activation='relu')`**: Una capa densa con 64 neuronas y activación ReLU se utiliza como capa intermedia para capturar relaciones no lineales entre los embeddings de los ingredientes.
- **`Dense(num_classes, activation='softmax')`**: La capa de salida es una capa densa con activación softmax, que produce una probabilidad para cada una de las clases. Esto es adecuado para problemas de clasificación multiclas.

**Compilación del Modelo**:
- **`optimizer='adam'`**: Se utiliza el optimizador Adam, conocido por su eficacia y capacidad de adaptación durante el entrenamiento.
- **`loss='categorical_crossentropy'`**: La función de pérdida es la entropía cruzada categórica, que es adecuada para problemas de clasificación con múltiples clases.
- **`metrics=['accuracy']`**: Se ha especificado la precisión como métrica para evaluar el rendimiento del modelo durante el entrenamiento y la validación.

Como se ha mencionado ya con anterioridad, se busca que el modelo sea capaz de clasificar recetas con la más alta precisión, por lo que se realizarán pruebas posteriores con diferentes configuraciones de hiperparámetros y cambios en la arquitectura del modelo para buscar optimizar su rendimiento. A pesar de justificar arriba por qué se eligieron estos hiperparámetros, aquí comparto otras ideas con las que se experimentará y se determinará posteriormente cual da mejores resultados.

**Hiperparámetros a Experimentar**:
1. **Unidades LSTM**: Probar con diferentes cantidades de unidades LSTM, como 64 o 256, para ver cómo afecta la capacidad del modelo para aprender las relaciones entre ingredientes.
2. **Tasa de Dropout**: Ajustar la tasa de Dropout a valores como 0.3 o 0.7 para observar su impacto en la regularización y capacidad del modelo para generalizar.
3. **Optimizador**: Experimentar con otros optimizadores, como RMSprop o SGD, para determinar si pueden ofrecer mejores resultados en términos de convergencia y precisión.


In [117]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_train, y_train)
print(f"Training Accuracy: {accuracy:.4f}")

Epoch 1/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.4841 - loss: 1.8364 - val_accuracy: 0.6449 - val_loss: 1.1489
Epoch 2/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.6339 - loss: 1.2254 - val_accuracy: 0.6752 - val_loss: 1.0385
Epoch 3/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6601 - loss: 1.1440 - val_accuracy: 0.6878 - val_loss: 0.9985
Epoch 4/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.6776 - loss: 1.0889 - val_accuracy: 0.6963 - val_loss: 0.9839
Epoch 5/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.6838 - loss: 1.0611 - val_accuracy: 0.7082 - val_loss: 0.9523
Epoch 6/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.6989 - loss: 1.0218 - val_accuracy: 0.7073 - val_loss: 0.9451
Epoch 7/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.7091 - loss: 0.9962 - val_accuracy: 0.7216 - val_loss: 0.9232
Epoch 8/20
796/796 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.7135 - loss: 0.9612 - 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Generar secuencias de embeddings para todas las recetas en el conjunto de prueba (test_df)
X_test = np.array([get_embedding_sequence(recipe, fasttext_model) for recipe in test_df['ingredients']])

# Realizar las predicciones sobre el conjunto de prueba
predictions = model.predict(X_test)

# Convertir las predicciones en las clases de cocina (índices a etiquetas)
predicted_labels = np.argmax(predictions, axis=1)

# Obtener las categorías de cocina del modelo de entrenamiento
cuisine_labels = train_df['cuisine'].cat.categories

# Mapear los índices de las predicciones a las etiquetas de cocina
predicted_cuisines = cuisine_labels[predicted_labels]

# Comparar con las etiquetas reales en test_df
true_cuisines = test_df['cuisine']

# Calcular la exactitud (accuracy)
accuracy = accuracy_score(true_cuisines, predicted_cuisines)
print(f'Accuracy: {accuracy:.2f}')

# Generar un reporte de clasificación con precisión, recall y F1-score
report = classification_report(true_cuisines, predicted_cuisines)
print('Classification Report:')
print(report)


249/249 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
Accuracy: 0.74
Classification Report:
              precision    recall  f1-score   support

   brazilian       0.46      0.44      0.45        84
     british       0.30      0.22      0.25       157
cajun_creole       0.77      0.69      0.73       328
     chinese       0.74      0.85      0.79       510
    filipino       0.56      0.49      0.52       136
      french       0.48      0.65      0.55       550
       greek       0.71      0.60      0.65       249
      indian       0.88      0.87      0.88       602
       irish       0.48      0.30      0.37       151
     italian       0.82      0.84      0.83      1567
    jamaican       0.70      0.48      0.57        91
    japanese       0.78      0.66      0.72       284
      korean       0.71      0.65      0.68       166
     mexican       0.90      0.92      0.91      1336
    moroccan       0.73      0.71      0.72       166
     russian       0.56      0.22      0.32        89
 

### Ejemplo de los primeros 20 resultados

| Index  | True Cuisine   | Predicted Cuisine |
|--------|-----------------|-------------------|
| 21513  | chinese        | chinese           |
| 1796   | spanish        | spanish           |
| 21861  | greek          | greek             |
| 26571  | indian         | indian            |
| 28720  | italian        | italian           |
| 39404  | mexican        | mexican           |
| 29512  | italian        | italian           |
| 5726   | cajun_creole   | southern_us       |
| 35755  | greek          | greek             |
| 35346  | mexican        | mexican           |
| 27662  | cajun_creole   | southern_us       |
| 37335  | spanish        | southern_us       |
| 2116   | southern_us    | italian           |
| 8409   | chinese        | chinese           |
| 7672   | greek          | french            |
| 17060  | mexican        | mexican           |
| 6053   | indian         | southern_us       |
| 25959  | filipino       | chinese           |
| 7989   | mexican        | mexican           |
| 38987  | italian        | italian           |


### Resultados y Evaluación del Primer Modelo de Clasificación de Recetas

**Training Accuracy**: 0.7978

**Test Accuracy**: 0.74

**Interpretación de Resultados**:
El modelo de clasificación de recetas alcanzó una precisión de entrenamiento del 79.78%, lo que indica un buen ajuste durante el entrenamiento. Sin embargo, la precisión en el conjunto de prueba es del 74%, lo cual sugiere que el modelo podría beneficiarse de mayor refinamiento y ajustes de hiperparámetros para mejorar su capacidad de generalización.

**Reporte de Clasificación**:
El reporte de clasificación proporciona detalles sobre la precisión, recall y F1-score para cada una de las clases (tipos de cocina). Algunas observaciones claves incluyen:

- **Cocina Brasileña**: Tiene una precisión y recall relativamente bajos, lo que indica que el modelo tiene dificultades para identificar correctamente las recetas brasileñas.
- **Cocina Italiana y Mexicana**: Muestran altas precisiones y F1-scores, indicando que el modelo es capaz de clasificar estas cocinas de manera efectiva.
- **Cocina Británica y Rusa**: Muestran resultados más bajos en todas las métricas, sugiriendo que el modelo tiene dificultades con estas categorías.

In [122]:
# Guardar el modelo en disco
model.save("rnn_cuisine_classifier_v1.h5")

### Mejora del modelo 

Como se mencionó anteriormente, se realizarán pruebas posteriores con diferentes configuraciones de hiperparámetros y cambios en la arquitectura del modelo para buscar optimizar su rendimiento. A pesar de justificar arriba por qué se eligieron estos hiperparámetros, aquí comparto otras ideas con las que se experimentará y se determinará posteriormente cual da mejores resultados. 

**Hiperparámetros a Experimentar**:
- max_len = [11, 14] 
- LSTM = [64, 128] 
- Dropout = [0.3, 0.5] 
- optimizer = ['adam', 'sgd']

Para realizar lo anterior se aplicará la técnica de GridSearchCV, la cual permite probar diferentes combinaciones de hiperparámetros y seleccionar la mejor configuración para el modelo. Aquí a continuación se muestra como es que se modificó el modelo para poder realizar esto.

In [124]:
# Convertir las etiquetas de train y test en formato one-hot
y_train_one_hot = tf.keras.utils.to_categorical(train_df['cuisine'].cat.codes, num_classes=num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(test_df['cuisine'].cat.codes, num_classes=num_classes)

In [127]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import ParameterGrid
import numpy as np

# Definir los hiperparámetros a probar
param_grid = {
    'max_len': [11, 14],
    'lstm_units': [64, 128],
    'dropout_rate': [0.3, 0.5],
    'optimizer': ['adam', 'sgd']
}

# Crear una función para construir el modelo con los hiperparámetros dados
def create_model(max_len, lstm_units, dropout_rate, optimizer):
    model = Sequential([
        Masking(mask_value=0.0, input_shape=(max_len, embedding_dim)),
        LSTM(lstm_units, return_sequences=False),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    
    if optimizer == 'adam':
        opt = Adam()
    elif optimizer == 'sgd':
        opt = SGD()
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Preparar los datos de entrenamiento y prueba con las diferentes longitudes de secuencia
def preprocess_data(max_len):
    X_train_processed = np.array([get_embedding_sequence(recipe, fasttext_model, max_len=max_len) for recipe in train_df['ingredients']])
    X_test_processed = np.array([get_embedding_sequence(recipe, fasttext_model, max_len=max_len) for recipe in test_df['ingredients']])
    return X_train_processed, X_test_processed

# Convertir las etiquetas de train y test en formato one-hot
y_train_one_hot = tf.keras.utils.to_categorical(train_df['cuisine'].cat.codes, num_classes=num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(test_df['cuisine'].cat.codes, num_classes=num_classes)

# Realizar la búsqueda en cuadrícula
best_accuracy = 0
best_params = None
results = []

for params in ParameterGrid(param_grid):
    max_len = params['max_len']
    lstm_units = params['lstm_units']
    dropout_rate = params['dropout_rate']
    optimizer = params['optimizer']
    
    X_train_processed, X_test_processed = preprocess_data(max_len)
    
    model = create_model(max_len, lstm_units, dropout_rate, optimizer)
    history = model.fit(X_train_processed, y_train_one_hot, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    
    loss, accuracy = model.evaluate(X_test_processed, y_test_one_hot, verbose=0)
    print(f"Parameters: {params} => Accuracy: {accuracy:.4f}")
    results.append((params, accuracy))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

# Mostrar los resultados
print(f"Best Accuracy: {best_accuracy:.4f} with parameters: {best_params}")


Parameters: {'dropout_rate': 0.3, 'lstm_units': 64, 'max_len': 11, 'optimizer': 'adam'} => Accuracy: 0.7317
Parameters: {'dropout_rate': 0.3, 'lstm_units': 64, 'max_len': 11, 'optimizer': 'sgd'} => Accuracy: 0.6755
Parameters: {'dropout_rate': 0.3, 'lstm_units': 64, 'max_len': 14, 'optimizer': 'adam'} => Accuracy: 0.7443
Parameters: {'dropout_rate': 0.3, 'lstm_units': 64, 'max_len': 14, 'optimizer': 'sgd'} => Accuracy: 0.6885
Parameters: {'dropout_rate': 0.3, 'lstm_units': 128, 'max_len': 11, 'optimizer': 'adam'} => Accuracy: 0.7327
Parameters: {'dropout_rate': 0.3, 'lstm_units': 128, 'max_len': 11, 'optimizer': 'sgd'} => Accuracy: 0.6850
Parameters: {'dropout_rate': 0.3, 'lstm_units': 128, 'max_len': 14, 'optimizer': 'adam'} => Accuracy: 0.7393
Parameters: {'dropout_rate': 0.3, 'lstm_units': 128, 'max_len': 14, 'optimizer': 'sgd'} => Accuracy: 0.6950
Parameters: {'dropout_rate': 0.5, 'lstm_units': 64, 'max_len': 11, 'optimizer': 'adam'} => Accuracy: 0.7198
Parameters: {'dropout_rate':

### Interpretación de los Resultados del Modelo

El proceso de ajuste y evaluación del modelo utilizando diferentes configuraciones de hiperparámetros ha revelado importantes insights sobre el desempeño del modelo de clasificación de recetas. Los resultados obtenidos demuestran variaciones en la precisión del modelo dependiendo de los valores de los hiperparámetros probados.

**Comparación de Hiperparámetros y Métricas de Evaluación**:
1. **Max Len**:
   - Las longitudes de secuencia probadas fueron 11 y 14. Se observó que una longitud de secuencia de 14 proporcionó mejores resultados en comparación con una longitud de 11, lo cual es coherente con la necesidad de capturar suficiente información contextual de los ingredientes.
2. **Unidades LSTM**:
   - Se probaron 64 y 128 unidades LSTM. Las configuraciones con 64 unidades LSTM mostraron un rendimiento competitivo con configuraciones de 128 unidades, pero con menor complejidad computacional.
3. **Dropout Rate**:
   - Las tasas de dropout de 0.3 y 0.5 fueron comparadas. Se encontró que una tasa de dropout de 0.3 ofreció mejores resultados en la mayoría de los casos, posiblemente debido a un balance más efectivo entre regularización y retención de información útil.
4. **Optimizador**:
   - Se probaron los optimizadores Adam y SGD. Adam consistentemente superó a SGD en términos de precisión, lo cual es esperado debido a la capacidad adaptativa de Adam para ajustar las tasas de aprendizaje.

**Resultados Finales**:
El mejor desempeño del modelo se obtuvo con la siguiente configuración de hiperparámetros:
- **max_len**: 14
- **lstm_units**: 64
- **dropout_rate**: 0.3
- **optimizer**: Adam

Esta configuración alcanzó una precisión de **0.7443**, mostrando su efectividad en la clasificación de recetas basadas en ingredientes.

### Elección de los Parámetros Óptimos

La selección de los parámetros óptimos se basó en un cuidadoso análisis de los resultados de las pruebas. La longitud de secuencia de 14 fue elegida porque capturó una cantidad suficiente de información contextual sobre los ingredientes sin introducir demasiado ruido de ingredientes irrelevantes. Esta longitud equilibrada permitió al modelo aprender de manera efectiva las relaciones entre los ingredientes, hay muchos platillos que se quedan con ingredientes sin capturar debido a tener 11 de máximo, número que reduce bastante a comparación de con 14.

El número de unidades LSTM se estableció en 64, lo cual resultó ser suficiente para modelar las secuencias de ingredientes sin añadir una complejidad innecesaria al modelo. Esta elección ayudó a mejorar la eficiencia computacional y reducir el tiempo de entrenamiento sin sacrificar la precisión.

La tasa de dropout de 0.3 proporcionó el mejor balance entre regularización y retención de información, permitiendo al modelo generalizar bien sin sobreajustarse a los datos de entrenamiento. Esto es crucial para asegurar que el modelo funcione bien en datos no vistos.

El optimizador Adam fue seleccionado debido a su capacidad adaptativa y eficiencia en la convergencia del modelo, lo que permitió alcanzar una precisión más alta en comparación con SGD. Adam ajusta dinámicamente las tasas de aprendizaje, lo cual es especialmente útil para modelos complejos y grandes volúmenes de datos, que considerando la cantidad de ingredientes únicos que existían considero que en eso se vio mucho más beneficiado a comparación del SDG.

### Conclusión

**Conclusión Técnica**:
Los parámetros "ganadores" fueron seleccionados debido a su impacto positivo en el rendimiento del modelo. La longitud de secuencia de 14 permite capturar suficientes ingredientes para una representación robusta. Un menor número de unidades LSTM (64) ofrece un buen equilibrio entre rendimiento y eficiencia computacional. La tasa de dropout de 0.3 ayuda a regularizar el modelo sin perder demasiada información. Finalmente, el optimizador Adam proporciona una convergencia rápida y eficiente.

**Conclusión del Proyecto**:
Lo que más me gustó del proyecto fue la aplicación práctica del deep learning para resolver un problema real de clasificación de recetas, demostrando cómo la tecnología puede transformar datos complejos en información valiosa. Aprendí mucho sobre el ajuste de hiperparámetros y la importancia de probar diferentes configuraciones para optimizar el rendimiento del modelo. Los mayores retos fueron manejar la variabilidad de los datos de ingredientes y asegurarse de que el modelo no se sobreajustara. Estos retos fueron superados mediante una cuidadosa selección de hiperparámetros y el uso de técnicas de regularización como el dropout.

Este proyecto no solo enriqueció mi comprensión de las RNN y su aplicación, sino que también me proporcionó una experiencia invaluable en la práctica de machine learning y el ajuste de modelos.

---
